# INDENTATION 2D EXAMPLE WITH ARGIOPE AND HARDNESS 

In [1]:
%load_ext autoreload
%autoreload 2

In [16]:
import matplotlib.pyplot as plt
import hardness as hd
import argiope as ag
import os, subprocess, time, local_settings, time
%matplotlib nbagg

# USEFUL FUNCTIONS
def create_dir(path):
  try:
    os.mkdir(path)
  except:
    pass

## Settings

In [17]:
# SETTINGS
workdir   = "workdir/"
outputdir = "outputs/"
label   = "indentation_2D"

create_dir(workdir)
create_dir(workdir + outputdir)     

## Model definition

In [119]:
#-------------------------------------------------------------------------------
# MESH DEFINITIONS
def element_map(mesh):
    mesh.elements.loc[mesh.elements.type.argiope.x == "tri3", ("type", "solver", mesh._null)] = "CAX3" 
    mesh.elements.loc[mesh.elements.type.argiope.x == "quad4", ("type", "solver", mesh._null)] = "CAX4" 
    return mesh
    
    
def sample_material_map(mesh):
    mesh.elements["materials"] = "SAMPLE_MAT" 
    return mesh

def indenter_material_map(mesh):
    mesh.elements["materials"] = "INDENTER_MAT" 
    return mesh
    
    
meshes = {
    "sample" : hd.models.sample_mesh_2D("gmsh", 
                                   workdir, 
                                   lx = 1., ly = 1., 
                                   r1 = 2., r2 = 100., 
                                   Nx = 16, Ny = 16, 
                                   Nr = 2, Nt = 32,
                                   element_map = element_map,
                                   material_map = sample_material_map),
                                   
    "indenter" : hd.models.spheroconical_indenter_mesh_2D("gmsh", 
                                   workdir, 
                                   R = 1.,
                                   psi= 30., 
                                   r1 = 1., 
                                   r2 = 10., 
                                   r3 = 10., 
                                   lc1 = .1, 
                                   lc2 = 20.,
                                   rigid = False,
                                   element_map = element_map,
                                   material_map = indenter_material_map)}
                                   
materials = [ag.materials.Hollomon(label = "SAMPLE_MAT", strain_data_points = 10),
             ag.materials.Hollomon(label = "INDENTER_MAT", strain_data_points = 10)]
    
"""
sample_mesh.save(h5path = workdir + outputdir + simName + "_sample_mesh.h5")
indenter_mesh.save(h5path = workdir + outputdir + simName + "_indenter_mesh.h5")  
"""   
#-------------------------------------------------------------------------------
# STEP DEFINTIONS
steps = [
        hd.models.indentation_2D_step_input(name = "LOADING1",
                                            control_type = "disp", 
                                            duration = 1.,
                                            kind = "adaptative",  
                                            nframes = 50,
                                            controlled_value = -0.1),
        hd.models.indentation_2D_step_input(name = "UNLOADING1",
                                            control_type = "force", 
                                            duration = 1.,
                                            kind = "adaptative",  
                                            nframes = 50,
                                            controlled_value = 0.),
        hd.models.indentation_2D_step_input(name = "RELOADING1",
                                            control_type = "disp", 
                                            duration = 1.,
                                            kind = "adaptative",  
                                            nframes = 50,
                                            controlled_value = -0.1),
        hd.models.indentation_2D_step_input(name = "LOADING2",
                                            control_type = "disp", 
                                            duration = 1.,
                                            kind = "adaptative",  
                                            nframes = 50,
                                            controlled_value = -0.2),                                    
        hd.models.indentation_2D_step_input(name = "UNLOADING2",
                                            control_type = "force",
                                            kind = "adaptative", 
                                            duration = 1., 
                                            nframes = 50,
                                            controlled_value = 0.)
        ]                                                                                                  
#-------------------------------------------------------------------------------
"""                                  
hd.models.indentation_2D_input(sample_mesh   = sample_mesh,
                            indenter_mesh = indenter_mesh,
                            steps = steps ,
                            path = workdir + simName + ".inp")
                                     
hd.postproc.indentation_abqpostproc(path = workdir + simName + "_abqpp.py")        
hd.postproc.indentation_pypostproc(path = workdir + simName + "_pypp.py") 
"""
#-------------------------------------------------------------------------------
model = hd.models.Indentation2D(label = label, 
                      meshes = meshes, 
                      steps = steps, 
                      materials = materials, 
                      solver = "abaqus", 
                      solver_path = local_settings.ABAQUS_PATH,
                      workdir = workdir,
                      verbose = True)

model.write_input()
model.run_simulation()
model.write_postproc()
model.run_postproc()

<Running "indentation_2D" using abaqus>
<Ran indentation_2D: duration 48.50s>
<Post-Processing"indentation_2D" using abaqus>
(b'', None)
<Post-Processed indentation_2D: duration 2.65s>


In [90]:
mesh = meshes["sample"]
print(mesh.write_inp())

********************************************************************************
** NODES ***********************************************************************
********************************************************************************
*NODE
  1, 0.0, 0.0, 0.0
  2, 0.0, -1.0, 0.0
  3, 1.0, -1.0, 0.0
  4, 1.0, 0.0, 0.0
  5, 0.0, -2.0, 0.0
  6, 2.0, 0.0, 0.0
  7, 0.0, -100.0, 0.0
  8, 100.0, 0.0, 0.0
  9, 0.0, -0.4999999999986921, 0.0
  10, 0.4999999999986921, -1.0, 0.0
  11, 1.0, -0.5000000000020595, 0.0
  12, 0.5000000000020595, 0.0, 0.0
  13, 0.0, -1.194048043484349, 0.0
  14, 0.0, -1.4214187966830047, 0.0
  15, 0.0, -1.6878344506928291, 0.0
  16, 0.7653668647282327, -1.84775906502338, 0.0
  17, 1.4142135623704117, -1.4142135623757779, 0.0
  18, 1.8477590650219262, -0.765366864731742, 0.0
  19, 1.194048043484349, 0.0, 0.0
  20, 1.4214187966830047, 0.0, 0.0
  21, 1.6878344506928291, 0.0, 0.0
  22, 0.0, -22.92214864262552, 0.0
  23, 38.268343236407674, -92.38795325117063, 0.0
  2

In [91]:
meshes["sample"].elements.sets

,SHELL2,SHELL1,CORE,ALL_ELEMENTS,_SURF_SURFACE_FACE2,_SURF_SURFACE_FACE3,_SURF_SURFACE_FACE4,_MAT_SAMPLE_MAT
,x,x,x,x,x,x,x,x
element,,,,,,,,
19,False,True,False,True,False,True,False,True
20,False,True,False,True,True,False,False,True
21,False,False,True,True,False,False,True,True
22,False,False,True,True,False,False,True,True
23,False,False,True,True,False,False,False,True
24,False,False,True,True,False,False,False,True
25,False,True,False,True,False,False,False,True
26,False,True,False,True,False,False,False,True


In [116]:
materials[0].write_inp()


'********************************************************************************\n** MATERIAL CLASS: Hollomon / LABEL: SAMPLE_MAT\n********************************************************************************\n** Poisson_ratio = 0.3\n** Young_modulus = 1.0\n** hardening_exponent = 0.2\n** label = SAMPLE_MAT\n** max_strain = 10.0\n** strain_data_points = 10\n** yield_stress = 0.01\n*MATERIAL, NAME = SAMPLE_MAT\n*ELASTIC\n  1.0, 0.3\n*PLASTIC\n0.01,0.0\n0.011659144011798317,0.009885202888520522\n0.013593563908785257,0.03282232442734253\n0.015848931924611138,0.08415106807538887\n0.01847849797422291,0.19696497102896543\n0.02154434690031884,0.44261453646095894\n0.0251188643150958,0.9748811356849042\n0.029286445646252362,2.12514824438563\n0.03414548873833602,4.607443344874442\n0.03981071705534973,9.96018928294465'